# 9章 潜在顧客を把握するための画像認識

ここでは、カメラから取得した映像を用いて画像認識を行い、
必要な情報を取得するための流れを学ぶことで、
画像認識をビジネス現場で応用するイメージをつかみます。

画像 Data を表示する

In [6]:
import cv2

img = cv2.imread('sample_code/chapter_9/img/img01.jpg')
height, width = img.shape[:2]
print(f'画像幅: {str(width)}')
print(f'画像高さ: {str(height)}')
cv2.imshow("img", img)
cv2.waitKey(3000)
cv2.destroyAllWindows()

画像幅: 1920
画像高さ: 1440


映像 Data を表示する

In [7]:
import cv2

# 情報取得
cap = cv2.VideoCapture('sample_code/chapter_9/mov/mov01.avi')
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
fps = cap.get(cv2.CAP_PROP_FPS)
print(f'画像幅: {str(width)}')
print(f'画像高さ: {str(height)}')
print(f'総フレーム数: {str(count)}')
print(f'FPS: {str(fps)}')

# 出力
while (cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()

cv2.destroyAllWindows()

画像幅: 1920.0
画像高さ: 1440.0
総フレーム数: 401.0
FPS: 30.0


映像を画像に分割し、保存する

In [4]:
import cv2

cap = cv2.VideoCapture('sample_code/chapter_9/mov/mov01.avi')
num = 0
while (cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        cv2.imshow('frame', frame)
        filepath = f'snapshot/snapshot_{str(num)}.jpg'
        cv2.imwrite(filepath, frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    num = num + 1

cap.release()
cv2.destroyAllWindows()